# Connect to Elasticsearch


In [1]:
from pprint import pprint 
from elasticsearch import Elasticsearch

es = Elasticsearch(
    "https://localhost:9200", 
    basic_auth=("elastic", "6AqhOxi*CPXYvCZl7Iln"), 
    verify_certs=False)
client_info = es.info() 
print("Connected to Elasticsearch!")
pprint(client_info.body)

c:\Users\Asus\Desktop\sastokinmel\venv\Lib\site-packages\elasticsearch\_sync\client\__init__.py:313: SecurityWarning: Connecting to 'https://localhost:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(
c:\Users\Asus\Desktop\sastokinmel\venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Connected to Elasticsearch!
{'cluster_name': 'docker-cluster',
 'cluster_uuid': 'mIJwhjTmStW54eKFEwQnMA',
 'name': 'f12c85f397e4',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2026-01-29T10:05:46.708397977Z',
             'build_flavor': 'default',
             'build_hash': '17b451d8979a29e31935fe1eb901310350b30e62',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '10.3.2',
             'minimum_index_compatibility_version': '8.0.0',
             'minimum_wire_compatibility_version': '8.19.0',
             'number': '9.3.0'}}


# ILM policy 

In [2]:
from pprint import pprint

policy = {
    'phases': {
        'hot': {
            'actions': {
                'rollover': {
                    'max_age': '5m', 
                }
            }
        }, 
        "delete": {
            "min_age": "20m", 
            "actions": {
                "delete": {}
            }
        }
    }
}

response = es.ilm.put_lifecycle(name="cpu_usage_policy_v2", policy=policy)
pprint(response.body)

c:\Users\Asus\Desktop\sastokinmel\venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'acknowledged': True}


# Index Template 

In [3]:

response = es.indices.put_index_template(
    name="cpu_example_template",
    index_patterns=[
        "cpu_example_template*"  # Applies to any index starting with 'cpu_usage_data'
    ],
    data_stream={},
    template={
        "settings": {
            "index.mode": "time_series",  # Enable time series data mode
            "index.lifecycle.name": "cpu_usage_policy_v2",  # Apply the ILM policy
        },
        "mappings": {
            "properties": {
                "@timestamp": {
                    "type": "date"
                },
                "cpu_usage": {
                    "type": "float",
                    "time_series_metric": "gauge"
                },
                "cpu_name": {
                    "type": "keyword",
                    "time_series_dimension": True
                }
            }
        }
    },
    priority=500,  # A priority higher than 200 to avoid collisions with built-in templates
    meta={
        "description": "Template for CPU usage data",
    },
    allow_auto_create=True
)
pprint(response.body)

c:\Users\Asus\Desktop\sastokinmel\venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'acknowledged': True}


In [4]:
import psutil
from datetime import datetime


index_alias = "cpu_example_template"
while True:
    document = {
        "@timestamp": datetime.utcnow(),
        "cpu_usage": psutil.cpu_percent(interval=0.1),
        "cpu_name": "i7-13650HX"
    }

    es.index(index=index_alias, document=document, refresh=True)

C:\Users\Asus\AppData\Local\Temp\ipykernel_4584\1949228889.py:8: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "@timestamp": datetime.utcnow(),
c:\Users\Asus\Desktop\sastokinmel\venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\Asus\Desktop\sastokinmel\venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\Asus\Desktop\sastokinmel\venv\Lib\site-packa

ConnectionTimeout: Connection timed out